In [1]:
from langchain_ollama import OllamaEmbeddings
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain import hub
import os
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from dotenv import load_dotenv
from langchain_core.documents import Document

from langchain_openai import ChatOpenAI
load_dotenv()



True

In [2]:

QDRANT_ARCH_LINUX_URL = os.environ["QDRANT_ARCH_LINUX_URL"]
LANGSMITH_TRACING=True
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="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"
LANGSMITH_PROJECT="pr-linear-offense-26"

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")


In [3]:
embeddings = OllamaEmbeddings(model="llama3.2")
client = QdrantClient(url=QDRANT_ARCH_LINUX_URL)
vector_store = QdrantVectorStore(client=client, embedding=embeddings, collection_name="legal_docs_ollama")
llm = ChatOpenAI(model = "gpt-4o-mini", api_key=OPENAI_API_KEY)
#test_prompt = "What are the things to be considered in invoking and proving psychological incapacity?"
#test_docs = vector_store.similarity_search(test_prompt)


In [4]:
prompt = hub.pull("rlm/rag-prompt",api_key=LANGSMITH_API_KEY)
example_messages = prompt.invoke({"context":"(context goes here)", "question":"(question goes here)"}).to_messages()
assert len(example_messages)==1
print(example_messages[0].content)


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


In [5]:
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"],k=10)
    print(f"retrieved {len(retrieved_docs)} documents")
    print("****")
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response0 = llm.invoke(messages)
    return {"answer": response0.content}


graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

response = graph.invoke({"question": "What are the things to be considered in invoking and proving psychological incapacity?"})
print(response["answer"])


retrieved 10 documents
I don't know.
